# 1. **Business Problem**

* The Client

    XYZ is a private firm in US. Due to remarkable growth in the Cab Industry in last few years and multiple key players in the market, it is planning for an investment in Cab industry and as per their Go-to-Market(G2M) strategy they want to understand the market before taking final decision.

* Project Objective:

    To generate insights to help XYZ identify the right company to make their investment.

* In next sections we will analyze data in the following process:

    * Data Intake and Preliminary Prossess
    * Compare Yellow Cab and Pink Cab from different aspects
    * Recommendations for investment

# 2. Data Intake and Preliminary Prossess
First of all, we import all the packages that are required in this porject. 


In [1]:
# First of all, we import all the packages that are required in this porject. 
import numpy as np # linear algebra
import matplotlib.pyplot as plt
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from scipy import stats
import plotly.express as px
import plotly.graph_objs as go
import plotly
import plotly.graph_objects as go
import datetime

In [2]:
Cab_Data = pd.read_csv('/kaggle/input/cabs-fare-data/Cab_Data.csv')
Customer_Data = pd.read_csv('../input/cabs-fare-data/Customer_ID.csv')
Transaction_Data = pd.read_csv('../input/cabs-fare-data/Transaction_ID.csv')
City_Data = pd.read_csv('../input/cabs-fare-data/City.csv')
Holiday_Data = pd.read_csv('../input/us-holiday-dates-2004-2021/US Holiday Dates (2004-2021).csv')

In [3]:
Cab_Data.head(10)

In [4]:
Customer_Data.head(10)

In [5]:
Transaction_Data.head(10)

In [6]:
City_Data.head(10)

In [7]:
Holiday_Data.head(10)

In [8]:
Cab_Data.info()
Customer_Data.info()
Transaction_Data.info()
City_Data.info()
Holiday_Data.info()

In [9]:
print(Cab_Data.describe())
print(Customer_Data.describe())
print(Transaction_Data.describe())
print(City_Data.describe())
print(Holiday_Data.describe())

Data info tells us that there are 0 missing values in all 5 data sets. 

Next, we are going to merge data, change data type and create some interested features. 

In [10]:
# Prepare Holiday_Data for merging
Holiday_Data['Date'] = pd.to_datetime(Holiday_Data['Date'])
Holiday = [1]*Holiday_Data.shape[0]
Holiday_Data['Holiday'] = Holiday
Holiday_Data = Holiday_Data[['Date','Holiday']]
Holiday_Data.drop_duplicates()
Holiday_Data.info()


In [11]:
# Prepare Cab_data for merging
Cab_Data['Date of Travel'] = pd.to_datetime(Cab_Data['Date of Travel'])
Cab_Data = Cab_Data.rename(columns = {'Date of Travel': 'Date'})
Cab_Data.info()


In [12]:
# Merged all data into one data set for convenience.
Merged_Data = Cab_Data.merge(Transaction_Data, on= 'Transaction ID').merge(Customer_Data, on ='Customer ID').merge(City_Data, on = 'City').merge(Holiday_Data, how="left",on = 'Date')
Merged_Data['Holiday'] = Merged_Data['Holiday'].fillna(0)
missing_value = 0
for column in Merged_Data.columns :
    missing_value += Merged_Data[column].isna().sum() + Merged_Data[column].isnull().sum()
print('Merged dataset has {} entries , {} features and {} missing values'.format(Merged_Data.shape[0] ,Merged_Data.shape[1] ,missing_value))
print("\nFeature's datatypes\n\n{}".format(Merged_Data.dtypes))


In [13]:
# data cleaning
for column in Merged_Data.columns:
    if ' ' in column:
        Merged_Data = Merged_Data.rename(columns={column:column.replace(' ','_')})
for column in ["Population","Users"] :
    Merged_Data[column] = Merged_Data[column].str.replace(',','')
for column in ["Company", "City" , "Payment_Mode" , "Gender" ] :
    Merged_Data[column] = Merged_Data[column].astype('category')
for column in ["Population", "Users","Holiday"] :
    Merged_Data[column] = Merged_Data[column].astype('int64')
# Merged_Data['Holiday'] = Merged_Data['Holiday'].astype('bool')
Merged_Data.info()


In [14]:
# Derive some interested features for later analysis
Merged_Data['Year'] = Merged_Data['Date'].dt.year
Merged_Data['Month'] = Merged_Data['Date'].dt.month
Merged_Data['Profit'] = Merged_Data['Price_Charged'] - Merged_Data['Cost_of_Trip']
Merged_Data['Profit_Rate'] = ((Merged_Data['Price_Charged'] - Merged_Data['Cost_of_Trip'])/Merged_Data['Cost_of_Trip'])*100
Merged_Data['Profit_per_KM'] = Merged_Data['Profit']/ Merged_Data['KM_Travelled']
Merged_Data['User_Ratio'] = Merged_Data['Users']/Merged_Data['Population']*100
Merged_Data.info()


# 3. Compare Yellow Cab and Pink Cab from Different Aspects

In this section , we compare the data of Yellow Cab and Pink Cab to answer the following questions.

**3.1 What's the profit rate of Cab market? Which company has higher profit rate?**

**3.2 Does Yellow Cab have bigger market share than Pink Cab?**

**3.3 Do Customers prefer Yellow Cab to Pink Cab?**

**3.4 Who serves the higher income**

**3.5 Which company prevail in which city?**

**3.6 Can Yellow Cab make more profit in the future?**
 
As some figures and tables have implications for different questions, we will analyze step-by-step and then make a summary answer at last. 

In [15]:
fig,axes = plt.subplots(3, 2 , figsize=(26,12), sharey=True)
fig.suptitle('Distributions of Target Variables')
sns.histplot(ax=axes[0,0], data=Merged_Data , x='Cost_of_Trip' , kde = True , hue="Company")
sns.histplot(ax=axes[0,1], data=Merged_Data , x='Price_Charged' , kde = True , hue="Company")
sns.histplot(ax=axes[1,0], data=Merged_Data , x='Profit' , kde = True , hue="Company") 
sns.histplot(ax=axes[1,1], data=Merged_Data , x='Profit_Rate' , kde = True , hue="Company")
sns.histplot(ax=axes[2,0], data=Merged_Data , x='Profit_per_KM' , kde = True , hue="Company")

We can see from the last three figures that distributions of profits are all right skewed. But Yellow Cab has a visible bump to the right of peak in the distribution of Porfit Rate and Profit per KM, which will probably lead to higher total profit rate. So we will check company's profit rate next. 

In [16]:
Annual_Profit_Rate = Merged_Data.groupby(['Year','Company']).Profit.sum().unstack()/Merged_Data.groupby(['Year','Company']).Cost_of_Trip.sum().unstack()*100
Cab_Annual_PR = Merged_Data.groupby(['Year']).Profit.sum()/Merged_Data.groupby(['Year']).Cost_of_Trip.sum()*100
Annual_Profit_Rate['Cab Market'] = Cab_Annual_PR
print("Annual Profit Rate of Cab Market")
print(Annual_Profit_Rate)
ax = Annual_Profit_Rate.plot(kind='bar',stacked = False, title = 'Annual Profit Rate Comparison')
plt.show()

From the last figure and table, we can see that Yellow Cab does have higher yearly profit rate (48% - 56%) than Pink Cab(21%-27%) in each of the recorded year. And one of the reasons is Yellow Cab have more high-profit-rate rides proportionally than Pink Cab. From this point of view, we think Yellow Cab is more efficient, but we still need to mine deeper into the data for an overall investment recommendation. 

In [17]:
# Assume Pink and Yellow Cap are the only cab companies in USA. 
# Check market share of both cab companies at national and city level. 
Company_Transaction = Merged_Data.groupby(['City','Company']).Transaction_ID.count().unstack()
Company_Transaction.loc['All Cities'] = Company_Transaction.sum(numeric_only=True)
Company_Transaction['City_Market'] = Company_Transaction['Pink Cab'] + Company_Transaction['Yellow Cab']
Company_Transaction['Pink_Market_Share'] = Company_Transaction['Pink Cab']/Company_Transaction['City_Market']
Company_Transaction['Yellow_Market_Share'] = Company_Transaction['Yellow Cab']/Company_Transaction['City_Market']
ax = Company_Transaction.iloc[:,3:5].plot(kind='barh',stacked = False, title = 'Market Share Comparison across Cities')
plt.show()

Assume Pink and Yellow Cap are the only cab companies in USA. Yellow Cap's market share is three times greater than that of Pink Cap nationally. And Yellow Cap's advantage exists in most cites. As market share is highly related to the number of cabs owned by each company, this can be seen a stable advantage of Yellow Cab.

In [18]:
from sklearn.preprocessing import LabelEncoder
Data = Merged_Data.drop(['Transaction_ID', 'Customer_ID'], axis=1)
le = LabelEncoder()
Data["Gender"] = le.fit_transform(Data["Gender"])
Data["City"] = le.fit_transform(Data["City"])
Data["Payment_Mode"] = le.fit_transform(Data["Payment_Mode"])
Pink_Data = Data[Data['Company'] == 'Pink Cab'] 
Yellow_Data = Data[Data['Company'] == 'Yellow Cab']
Data["Company"] = le.fit_transform(Data["Company"])
Data

In [19]:
# Check out more distributions of variables using Boxplot
fig,axes = plt.subplots(2, 3, figsize=(25, 8), sharey=True)
fig.suptitle('Boxplot Distributions of the Variables')

sns.boxplot(ax=axes[0,0], x='KM_Travelled' ,        data=Pink_Data ).set_ylabel("Pink Cab")
sns.boxplot(ax=axes[0,1], x='Price_Charged'   ,     data=Pink_Data  ) .set_ylabel("Pink Cab")
sns.boxplot(ax=axes[0,2], x='Cost_of_Trip' ,        data=Pink_Data ).set_ylabel("Pink Cab")
sns.boxplot(ax=axes[1,0], x='KM_Travelled' ,        data=Yellow_Data ).set_ylabel("Yellow Cab")
sns.boxplot(ax=axes[1,1], x='Price_Charged'   ,     data=Yellow_Data  ) .set_ylabel("Yellow Cab")
sns.boxplot(ax=axes[1,2], x='Cost_of_Trip' ,        data=Yellow_Data ).set_ylabel("Yellow Cab")

In [20]:
fig,axes = plt.subplots(2, 3, figsize=(25, 8), sharey=True)
fig.suptitle('Boxplot Distributions of the Variables')
sns.boxplot(ax=axes[0,0], x='Profit' ,                 data=Pink_Data ).set_ylabel("Pink Cab")
sns.boxplot(ax=axes[0,1], x='Profit_Rate' ,  data=Pink_Data  ) .set_ylabel("Pink Cab")
sns.boxplot(ax=axes[0,2], x='Profit_per_KM' ,          data=Pink_Data ).set_ylabel("Pink Cab")
sns.boxplot(ax=axes[1,0], x='Profit' ,                 data=Yellow_Data ).set_ylabel("Yellow Cab")
sns.boxplot(ax=axes[1,1], x='Profit_Rate' ,  data=Yellow_Data  ) .set_ylabel("Yellow Cab")
sns.boxplot(ax=axes[1,2], x='Profit_per_KM' ,          data=Yellow_Data ).set_ylabel("Yellow Cab")

In [21]:
fig,axes = plt.subplots(2, 2, figsize=(25, 8), sharey=True)
fig.suptitle('Boxplot Distributions of the Variables')
sns.boxplot(ax=axes[0,0], x='Population' ,                 data=Pink_Data ).set_ylabel("Pink Cab")
sns.boxplot(ax=axes[0,1], x='Users' ,  data=Pink_Data  ) .set_ylabel("Pink Cab")
sns.boxplot(ax=axes[1,0], x='Population' ,                 data=Yellow_Data ).set_ylabel("Yellow Cab")
sns.boxplot(ax=axes[1,1], x='Users' ,  data=Yellow_Data  ) .set_ylabel("Yellow Cab")

From the boxplot of 'Population' and 'Users', Pink Cab serves mainly small cities while Yellow Cab has more transactions happened in big cities.

In [22]:
fig,axes = plt.subplots(2, 2, figsize=(25, 8), sharey=True)
fig.suptitle('Boxplot Distributions of the Variables')
sns.boxplot(ax=axes[0,0], x='Age' ,                 data=Pink_Data ).set_ylabel("Pink Cab")
sns.boxplot(ax=axes[0,1], x='Income_(USD/Month)' ,          data=Pink_Data ).set_ylabel("Pink Cab")
sns.boxplot(ax=axes[1,0], x='Age' ,                 data=Yellow_Data ).set_ylabel("Yellow Cab")
sns.boxplot(ax=axes[1,1], x='Income_(USD/Month)' ,          data=Yellow_Data ).set_ylabel("Yellow Cab")

In [23]:
Customer_Merged_Data = Cab_Data.merge(Transaction_Data, on= 'Transaction ID').merge(Customer_Data, on ='Customer ID')
Customer_Merged_Data = Customer_Merged_Data[['Company', 'Customer ID','Gender']]

In [24]:
Customer_Merged_Data = Cab_Data.merge(Transaction_Data, on= 'Transaction ID').merge(Customer_Data, on ='Customer ID')
Customer_Merged_Data = Customer_Merged_Data[['Company', 'Customer ID','Gender']]
Customer_Merged_Data = Customer_Merged_Data.drop_duplicates()
Customer_Gender = Customer_Merged_Data.groupby(['Company', 'Gender']).count().unstack()
Customer_Gender['Female_Rate'] = Customer_Gender.iloc[:,0] / (Customer_Gender.iloc[:,0] + Customer_Gender.iloc[:,1])
print(Customer_Gender)

From the boxplot of 'Age' and 'Income_(USD/Month)' and the table of Customer_Gender, customers of Pink Cab and Yellow Cab have similar statistics.

In [25]:
Customer_Merged_Data_2 = Cab_Data.merge(Transaction_Data, on= 'Transaction ID').merge(Customer_Data, on ='Customer ID')
Customer_Merged_Data_2['Year'] = Customer_Merged_Data_2['Date'].dt.year
Customer_Merged_Data_2 = Customer_Merged_Data_2[['Company', 'Customer ID','Year']]
Customer_Merged_Data_2 = Customer_Merged_Data_2.drop_duplicates()
Yellow_Customer_Merged_Data = Customer_Merged_Data_2[Customer_Merged_Data_2['Company'] == "Yellow Cab"].drop(['Company'], axis = 1).groupby(['Year']).count()
Pink_Customer_Merged_Data = Customer_Merged_Data_2[Customer_Merged_Data_2['Company'] == "Pink Cab"].drop(['Company'], axis = 1).groupby(['Year']).count()
Customer_Loyalty = Customer_Merged_Data_2.groupby(['Customer ID','Year']).count().unstack()
#tmp = Customer_Loyalty[Customer_Loyalty['Company' == "Yellow Cab"]]
print("Customer Number of Yellow Cab")
print(Yellow_Customer_Merged_Data)
print("Customer Number of Pink Cab")
print(Pink_Customer_Merged_Data)
seq = np.arange(0.1,1,0.1)
print(Customer_Loyalty.describe(seq))


Yellow Cap has more customers in all years and 60%-70% customers use only Cap company. 

In [26]:
Corr = Data.corr()
plt.figure(figsize=(20,10))
sns.heatmap(Corr,annot=True,vmin=-1,vmax=1,cmap='coolwarm')

In [27]:
#
Pink_Corr = Pink_Data.corr()
plt.figure(figsize=(20,10))
sns.heatmap(Pink_Corr,annot=True,vmin=-1,vmax=1,cmap='coolwarm')

In [28]:
#
Yellow_Corr = Yellow_Data.corr()
plt.figure(figsize=(20,10))
sns.heatmap(Yellow_Corr,annot=True,vmin=-1,vmax=1,cmap='coolwarm')

Heatmaps also tell us some intuitive truth: Price_Charged is positively related to profit indexes, i.e., Profit, Profit_Rate, Profit_per_KM. However, we also find that the correlations between Population, Users and profit indexes of Yellow Cab is much higher than those of Pink Cab. This review the pricing power of Yellow Cab in bigger cities. Or we can say that geographical price differentiation is the strategy Yellow Cab uses to increase profit.

Let's review the questions asked at the beginning of this section:

**3.1 What's the profit rate of Cab market? Which company has higher profit rate?**

**Ans: 50% is the profit of Cab Market, and Yellow Cab has higher yearly profit rate (48% - 56%) than Pink Cab(21%-27%) in each of the recorded year.**

**3.2 Does Yellow Cab have bigger market share than Pink Cab?**

**Ans: Yes, Yellow Cab's market share is two times greater than Pink Cab's, though customer numer is just 0.5 time greater.**


**3.3 Do Customers prefer Yellow Cab to Pink Cab?**

**Ans: Not very clear from the data.**

**3.4 Who serves the higher income**

**Ans:There isn't very high difference in customer characteristics of both companies.**

**3.5 Which company prevail in which city?**

**Ans:Yellow Cab prevails in most cities except San Diego, Sacramento, Pittsburgh, and Nashville.**

**3.6 Can Yellow Cab make more profit in the future?**

**Ans:There are only three years of data, we are not sure whether Yellow cab can keep up their profit indexes as there is a decreasing in 2018 compared to 2016 and 2017. But Yellow Cab should be able to continue to earn more profit than Pink Cab with dominant market share and better pricing strategy.**

# 4. Investment Recommendation

**According to the analysis in Section 3, we recommend XYZ company to invest in Yellow Cab.**